# Hyperparameter Search

## Setup

In [1]:
# Import packages 
import numpy as np

from tqdm import tqdm, trange 
import glob
import time

from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader, random_split, Dataset

import torchvision
import torchvision.transforms.functional as TF

import albumentations as A
from albumentations.pytorch import ToTensorV2

/pfs/data5/home/tu/tu_tu/tu_zxoau27/penv/lib64/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model 

In [2]:
# double convolutional layer which is executed in every step of the u-net 
# conv layer takes as input number of input channels -> in_channels and outputs vice versa
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    # forward pass in the conv layer 
    def forward(self, x):
        return self.conv(x)

# design complete u-net shape 
# model takes as default 3 input channels and 6 output channels
class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=6, features=[64, 128, 256, 512],  # features -> num of input nodes at every stage in the model 
    ):
        super(UNET, self).__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):  # reverse the features i.o. to move upwards in the model 
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))
        
        # lowest stage in u-net 
        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        # final conv layer: takes in 64 channels and outputs 1 channel by default 
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    # forward pass of the u-net model between stages 
    def forward(self, x):
        skip_connections = []  # red arrows in the model representation 

        for down in self.downs:
            x = down(x)  # one DoubleConv run-through 
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

## Import Data

In [4]:
%%script echo skipping 

# Establish connection with google drive
from google.colab import drive
drive.mount('/content/drive')

# Set path variable
path = "/content/drive/MyDrive/DS-Project/"

skipping


In [6]:
path = "/pfs/work7/workspace/scratch/tu_zxobe27-ds_project/"

In [7]:
# Create lists with paths to all image and label data 
imgs = glob.glob(path+'data/model_training/2_Ortho_RGB/sliced/*tif')
labels = glob.glob(path+"data/model_training/Labels_all/sliced/*tif")

# Create dictionary -> {key: 'link/to/image_or_label'}
labels_dict = {label.split("/")[-1].split(".")[0].rsplit('_', 1)[0] : label for label in labels}
imgs_dict = {img.split("/")[-1].split(".")[0].rsplit('_', 1)[0] : img for img in imgs}

# Create list with all keys 
keys = sorted(list(set(imgs_dict)))

## Custom Dataset 

In [8]:
class PotsdamDataset(Dataset):
    def __init__(self, imgs_dict, labels_dict, keys, transform=None):
        self.img_dir = imgs_dict
        self.mask_dir = labels_dict
        self.keys = keys
        self.transform = transform
        
        self.RGB_classes = {
            'imprevious' : [255, 255, 225],
            'building' : [0,  0, 255],
            'low_vegetation' : [0, 255, 255],
            'tree' : [0,  255,  0], 
            'car' : [ 255, 255, 0],
            'background' : [255, 0, 0]
            }  # in RGB
        
        self.bin_classes = ['imprevious', 'building', 'low_vegetation', 'tree', 'car', 'background']

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        img_path = self.img_dir[self.keys[idx]]
        mask_path = self.mask_dir[self.keys[idx]]
        
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("RGB"))
        
        cls_mask = np.zeros(mask.shape) # dim: (6000, 6000, 3)
        
        cls_mask[(mask == self.RGB_classes['imprevious']).all(-1)] = self.bin_classes.index('imprevious')
        cls_mask[(mask == self.RGB_classes['building']).all(-1)] = self.bin_classes.index('building')
        cls_mask[(mask == self.RGB_classes['low_vegetation']).all(-1)] = self.bin_classes.index('low_vegetation')
        cls_mask[(mask == self.RGB_classes['tree']).all(-1)] = self.bin_classes.index('tree')
        cls_mask[(mask == self.RGB_classes['car']).all(-1)] = self.bin_classes.index('car')
        cls_mask[(mask == self.RGB_classes['background']).all(-1)] = self.bin_classes.index('background')
        cls_mask = cls_mask[:,:,0] # omit last dimension (, , 3) -> RGB  

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=cls_mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask

## Util Functions

### Train-/Validation-Split 

In [9]:
dataset = PotsdamDataset(imgs_dict, labels_dict, keys)
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

### Data Loader

In [10]:
def get_loaders(
    imgs_dict,
    labels_dict,
    keys,
    batch_size,
    train_transform,
    val_transform,
    num_workers = 2,
    pin_memory = True,
):
    
    train_data = PotsdamDataset(
        imgs_dict = imgs_dict,
        labels_dict = labels_dict,
        keys = keys, 
        transform = train_transform,
    )
    
    valid_data = PotsdamDataset(
        imgs_dict = imgs_dict,
        labels_dict = labels_dict,
        keys = keys, 
        transform = val_transform,
    )

    train_loader = DataLoader(
        train_data,
        batch_size = batch_size,
        num_workers = num_workers,
        pin_memory = pin_memory,
        sampler = train_sampler,
    )

    val_loader = DataLoader(
        valid_data,
        batch_size = batch_size,
        num_workers = num_workers,
        pin_memory = pin_memory,
        sampler = valid_sampler
    )

    return train_loader, val_loader

### Transform Functions

In [11]:
def build_transforms(image_heigt, image_width): 
    
    train_transform = A.Compose([
        A.Resize(height=image_heigt, width=image_width),
        A.Flip(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), 
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
        ],)

    val_transform = A.Compose([
        A.Resize(height=image_heigt, width=image_width),
        A.Normalize(mean=(0.485, 0.456, 0.406), 
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
        ],)
    
    return train_transform, val_transform

### Accuracy Function 

In [12]:
def compute_accuracy(y_pred, y):

    preds = torch.argmax(y_pred, axis=1).cpu()
    num_correct = (preds == y).sum().item()
    num_pixels = torch.numel(preds)

    return num_correct/num_pixels

### Evaluation Function

In [13]:
def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0

    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            # compute probabilities
            probs = torch.nn.Softmax(model(x))
            # get predictions by choosing highest probability 
            preds = torch.argmax(probs.dim, axis=1).cpu()
            num_correct += (preds == y).sum().item()
            num_pixels += torch.numel(preds)
            
            #print(num_correct, num_pixels)

    print(
        f"Got {num_correct}/{num_pixels} pixels correct with acc {num_correct/num_pixels*100:.2f}"
    )

    return num_correct/num_pixels

In [14]:
def evaluate_fn(model, loader, criterion, device):

  epoch_loss = 0
  epoch_acc = 0

  model.eval()
  loop = tqdm(loader, desc="Evaluating", leave=False)

  with torch.no_grad():

    for x, y in loop:

      x = x.to(device)
      y = y.type(torch.LongTensor).to(device)

      y_pred = model(x)

      loss = criterion(y_pred, y)

      acc = compute_accuracy(y_pred, y.cpu())

      epoch_loss += loss.item()
      epoch_acc += acc

  return epoch_loss / len(loader), epoch_acc / len(loader)

In [15]:
def epoch_time(start_time, end_time):
  
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

## Configure Sweeps

In [16]:
# Setup wandb 
!pip install wandb --upgrade
import pprint
import wandb

wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 21.3 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8786 sha256=f24dffccf1bbca37d06c6af3375ff7d4a2d9bbb2bb5d7a7cb4c382d02af5c2b4
  Stored in directory: /pfs/data5/home/tu/tu_tu/tu_zxoau27/.cache/pip/wheels/4d/33/74/7c0903053e955973d5dc3d21857a29b3f8c0806ad0b05c32a1
Successfully built pathtools


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/tu/tu_tu/tu_zxoau27/.netrc


True

In [17]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'validation_accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

In [18]:
parameters_dict = {
    'batch_size': {
        'values': [8, 16, 32]
        },
    'learning_rate': {
        # a flat distribution
        'distribution': 'uniform',
        'min': 1e-5,
        'max': 5e-4
        },
    'epochs': {'values': [2, 4, 8]
        },
    }

sweep_config['parameters'] = parameters_dict


pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'validation_accuracy'},
 'parameters': {'batch_size': {'values': [8, 16, 32]},
                'epochs': {'values': [2, 4, 8]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.0005,
                                  'min': 1e-05}}}


In [19]:
# initialize sweep
sweep_id = wandb.sweep(sweep_config, project="ds_project")

Create sweep with ID: nof1txkn
Sweep URL: https://wandb.ai/ds_project/ds_project/sweeps/nof1txkn


## HP Search 

In [20]:
#def train_fn(loader, model, optimizer, criterion, scaler, device):
def train_fn(loader, model, optimizer, criterion, device):
  
  model.train()
  loop = tqdm(loader, desc="Training")
  
  for batch_idx, (data, targets) in enumerate(loop):
    data = data.to(device)
    targets = targets.type(torch.LongTensor).to(device)

    # forward
    # with torch.cuda.amp.autocast():
    with torch.autocast(device_type='cuda', dtype=torch.float16):
      predictions = model(data)
      loss = criterion(predictions, targets)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # backward + scaler
    # optimizer.zero_grad()
    # scaler.scale(loss).backward()
    # scaler.step(optimizer)
    # scaler.update()

    # update tqdm loop
    loop.set_postfix(loss=loss.item())

    torch.cuda.empty_cache()

In [21]:
IMAGE_HEIGHT = 300  # 2000 originally
IMAGE_WIDTH = 300  # 2000 originally

IMGS_DICT = imgs_dict
LABELS_DICT = labels_dict
KEYS = keys

NUM_WORKERS = 2
PIN_MEMORY = True
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [22]:
def main(config=None):
    
    with wandb.init(config=config):
        config = wandb.config
        
        train_transforms, val_transforms = build_transforms(
            image_heigt=IMAGE_HEIGHT,
            image_width=IMAGE_WIDTH
        )
        
        train_loader, validation_loader = get_loaders(
            imgs_dict=IMGS_DICT,
            labels_dict=LABELS_DICT,
            keys=KEYS,
            batch_size=config.batch_size,
            train_transform=train_transforms,
            val_transform=val_transforms,
            num_workers = NUM_WORKERS,
            pin_memory = PIN_MEMORY)
        
        # specify loss function 
        CRITERION = nn.CrossEntropyLoss().to(device=DEVICE)
        
        # initialize model and optimizer
        MODEL = UNET(in_channels=3, out_channels=6).to(device=DEVICE)
        OPTIMIZER = optim.Adam(MODEL.parameters(), lr=config.learning_rate)
        
        # SCALER = torch.cuda.amp.GradScaler()
        
        for epoch in range(config.epochs):
          print(f"Start epoch: {epoch+1}")
          
          train_fn(
              loader=train_loader, 
              model=MODEL, 
              optimizer=OPTIMIZER, 
              criterion=CRITERION, 
              #scaler=SCALER,
              device=DEVICE)
          
          # check accuracy
          validation_accuracy = check_accuracy(validation_loader, MODEL, device=DEVICE)
        
          print(f"End epoch: {epoch+1} \n")
        
          wandb.log({"epoch": epoch,
                     "validation_accuracy": validation_accuracy})

In [23]:
%%script echo skipping 

def main(config=None):
    
    with wandb.init(config=config):
        config = wandb.config
        
        train_transforms, val_transforms = build_transforms(
            image_heigt=IMAGE_HEIGHT,
            image_width=IMAGE_WIDTH
        )
        
        train_loader, validation_loader = get_loaders(
            imgs_dict=IMGS_DICT,
            labels_dict=LABELS_DICT,
            keys=KEYS,
            batch_size=config.batch_size,
            train_transform=train_transforms,
            val_transform=val_transforms,
            num_workers = NUM_WORKERS,
            pin_memory = PIN_MEMORY)
        
        # specify loss function 
        CRITERION = nn.CrossEntropyLoss().to(device=DEVICE)
        
        # initialize model and optimizer
        MODEL = UNET(in_channels=3, out_channels=6).to(device=DEVICE)
        OPTIMIZER = optim.Adam(MODEL.parameters(), lr=config.learning_rate)
        
        SCALER = torch.cuda.amp.GradScaler()

    for epoch in range(config.epochs):
        print(f"Start epoch: {epoch+1}")
        
        train_fn(
            loader=train_loader, 
            model=MODEL, 
            optimizer=OPTIMIZER, 
            criterion=CRITERION, 
            scaler=SCALER,
            device=DEVICE)
        
        # check accuracy
        validation_accuracy = check_accuracy(validation_loader, MODEL, device=DEVICE)
        
        print(f"End epoch: {epoch+1} \n")
        
        
    for epoch in trange(config.epochs, desc="Epochs"):

        start_time = time.monotonic()
        
        train_fn(
            loader=train_loader, 
            model=MODEL, 
            optimizer=OPTIMIZER, 
            criterion=CRITERION, 
            scaler=SCALER,
            device=DEVICE)
        
        training_loss, training_accuracy = evaluate_fn(
            model=MODEL, 
            loader=train_loader, 
            criterion=CRITERION, 
            device=DEVICE)
        
        validation_loss, validation_accuracy = evaluate_fn(
            model=MODEL, 
            loader=validation_loader, 
            criterion=CRITERION, 
            device=DEVICE)
                
        end_time = time.monotonic()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\t Train Loss: {training_loss:.3f} | Train Acc: {training_accuracy*100:.2f}%')
        print(f'\t Val. Loss: {validation_loss:.3f} |  Val. Acc: {validation_accuracy*100:.2f}%')
        
        wandb.log({"epoch": epoch, 
                   "validation_accuracy": validation_accuracy})

skipping


In [24]:
wandb.agent(sweep_id, main, count=10)

wandb: Agent Starting Run: wjyevld3 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 2
wandb: 	learning_rate: 0.0003640276738364003
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marvelo (ds_project). Use `wandb login --relogin` to force relogin


Start epoch: 1


Training: 100%|██████████| 18/18 [02:03<00:00,  6.87s/it, loss=1.11]


Got 2494180/6120000 pixels correct with acc 40.75
End epoch: 1 

Start epoch: 2


Training: 100%|██████████| 18/18 [01:42<00:00,  5.70s/it, loss=1.33]


...

Training: 100%|██████████| 9/9 [01:47<00:00, 11.91s/it, loss=1.6] 


Got 394823/6120000 pixels correct with acc 6.45
End epoch: 1 

Start epoch: 2


Training: 100%|██████████| 9/9 [01:48<00:00, 12.02s/it, loss=1.52]


Got 1202970/6120000 pixels correct with acc 19.66
End epoch: 2 



epoch,▁█
validation_accuracy,▁█
epoch,1
validation_accuracy,0.19656
